<a href="https://colab.research.google.com/github/marycx/ECE324-Project-Emoji-Predictor/blob/main/upload_BERT_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imporiting libraries

In [ ]:
!pip install -qq transformers
!pip install transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.0 MB/s eta 0:00:00


In [ ]:
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from transformers import AutoTokenizer
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8


RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
MAX_LEN = 200

In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Importing data

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# path = "/content/drive/MyDrive/Colab Notebooks/data_preproc_test.csv"
# path = "/content/drive/MyDrive/Colab Notebooks/ECE324/target_data_preproc_csv.csv"
path = "/content/drive/MyDrive/Colab Notebooks/ECE324/data_preproc_csv_50haha"
data = pd.read_csv(path)

In [ ]:
headerList = ['TEXT', 'Emoji']
data.to_csv("gfg2.csv", header=headerList, index=False)
data = pd.read_csv("gfg2.csv")

In [ ]:
data.head()

,TEXT,Emoji
0,Here to cure ur Sunday sads and bluezzz with #...,1
1,"KatyCats, get ready to own a piece of my histo...",2
2,It may be Oscar Sunday but dont worry theres s...,3
3,T-minus 1 month until Easter! get ur baskets r...,4
4,That feeling when the first styles from my Spr...,2


In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/ECE324/emoji_to_number_ref_50"
emoji_predicted = pd.read_csv(path)

In [ ]:

df = pd.DataFrame(emoji_predicted)
products_list = df.values.tolist()
print(products_list)

[[1, '💋'], [2, '😍'], [3, '😘'], [4, '😊'], [5, '✨'], [6, '🖤'], [7, '♥️'], [8, '🙏🏻'], [9, '‼️'], [10, '🔥'], [11, '🎉'], [12, '❤️'], [13, '😁'], [14, '🥰'], [15, '👏'], [16, '🙏'], [17, '🤩'], [18, '😂'], [19, '👀'], [20, '🏆'], [21, '🙏🏼'], [22, '🎥'], [23, '🤣'], [24, '😜'], [25, '👑'], [26, '😉'], [27, '🎄'], [28, '🚨'], [29, '💕'], [30, '😭'], [31, '🎂'], [32, '⚽️'], [33, '💜'], [34, '🤍'], [35, '📸'], [36, '💖'], [37, '💪'], [38, '➡️'], [39, '🙏🏾'], [40, '🎶'], [41, '🥳'], [42, '🙏🏽'], [43, '💥'], [44, '💯'], [45, '🙌'], [46, '😎'], [47, '👇'], [48, '💚'], [49, '👍']]


### Data Pre-Processing

In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'
# tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)


#### Showing the distribution of length of token

In [ ]:
token_lens = []
for txt in data.TEXT:
  tokens = tokenizer.encode(txt, max_length=512)
  token_lens.append(len(tokens))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<ipython-input-10-4dbe9bd8b56e>:1: UserWarning: 

`distplot` is a deprecated function and will be removed in seaborn v0.14.0.

Please adapt your code to use either `displot` (a figure-level function with
similar flexibility) or `histplot` (an axes-level function for histograms).

For a guide to updating your code to use the new functions, please see
https://gist.github.com/mwaskom/de44147ed2974457ad6372750bbe5751

  sns.distplot(token_lens)


In [ ]:
sns.distplot(token_lens)
plt.xlim([0, 256]);
plt.xlabel('Token count');

In [ ]:
MAX_LEN = 140

### Creating Data Set

In [ ]:
class GPReviewDataset(Dataset):
  def __init__(self, reviews, targets, tokenizer, max_len):
    self.reviews = reviews
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.reviews)
    
  def __getitem__(self, item):
    review = str(self.reviews[item])
    target = self.targets[item]
    encoding = self.tokenizer.encode_plus(
      review,
      add_special_tokens=True,
      max_length=self.max_len,
      return_token_type_ids=False,
      # pad_to_max_length=True,
      padding='max_length',
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt',
    )
    return {
      'review_text': review,
      'input_ids': encoding['input_ids'].flatten(),
      'attention_mask': encoding['attention_mask'].flatten(),
      'targets': torch.tensor(target, dtype=torch.long)
    }

### Splitting training, validation, and testing set

In [ ]:
df_train, df_test = train_test_split(
  data,
  test_size=0.3,
  random_state=RANDOM_SEED
)
df_val, df_test = train_test_split(
  df_test,
  test_size=0.5,
  random_state=RANDOM_SEED
)

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = GPReviewDataset(
    reviews=df.TEXT.to_numpy(),
    targets=df.Emoji.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )
  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=4
  )

In [ ]:
BATCH_SIZE = 16
train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
data_check = next(iter(train_data_loader))
data_check.keys()

dict_keys(['review_text', 'input_ids', 'attention_mask', 'targets'])

In [ ]:
bert_model = BertModel.from_pretrained("bert-base-cased", return_dict=False)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Building Classifier

In [ ]:
class SentimentClassifier(nn.Module):
  def __init__(self, n_classes):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained("bert-base-cased", return_dict=False)
    #add another layer before linear layer
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  def forward(self, input_ids, attention_mask):
    _, pooled_output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask, 
      return_dict=False
    )
    output = self.drop(pooled_output)
    return self.out(output)

In [ ]:
# MAX_EMOJI is the number of distinct emojis
MAX_EMOJI = 50
model = SentimentClassifier(MAX_EMOJI)
model = model.to(DEVICE)
input_ids = data_check["input_ids"].to(DEVICE)
attention_mask = data_check["attention_mask"].to(DEVICE)
print(input_ids.shape)
print(attention_mask.shape)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([16, 140])
torch.Size([16, 140])


### Training

#### define functions

In [ ]:
EPOCHS = 10
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=1e-4, correct_bias=False) #added weight_decay for L2 regularization
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


#### Building training function

In [ ]:
def train_epoch(
  model,
  data_loader,
  loss_fn,
  optimizer,
  device,
  scheduler,
  n_examples
):
  model = model.train()
  losses = []
  correct_predictions = 0
  for d in data_loader:
    input_ids = d["input_ids"].to(device)
    attention_mask = d["attention_mask"].to(device)
    targets = d["targets"].to(device)
    # input_ids = d["input_ids"]
    # attention_mask = d["attention_mask"]
    # targets = d["targets"]

    #forward pass
    outputs = model(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, targets)
    correct_predictions += torch.sum(preds == targets)
    losses.append(loss.item())

    #backprob
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
  return correct_predictions.double() / n_examples, np.mean(losses)

#### Define accuracy function

In [ ]:
def eval_model(model, data_loader, loss_fn, 
               device, 
               n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  one_correct_predictions = 0
  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )

      loss = loss_fn(outputs, targets)
      losses.append(loss.item())

      _, preds = torch.max(outputs, dim=1)
      one_correct_predictions += torch.sum(preds == targets)
 
      #if the prediction is within three classes, then it will be classified as correct
      topk_values, topk_indices = torch.topk(outputs, k=3, dim=1)
      num_columns = topk_indices.shape[1]

      for i in range(num_columns):
          column = topk_indices[:, i]
          correct_predictions = correct_predictions + torch.sum(column == targets)
 
  return correct_predictions.double() / n_examples, np.mean(losses), one_correct_predictions.double() / n_examples

  # return correct_predictions.double() / n_examples, np.mean(losses)

#### Training with epochs

In [ ]:
def make_pred(model, data_loader, loss_fn, 
               device, 
               n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  one_correct_predictions = 0
  with torch.no_grad():
    for d in data_loader:
      input_ids = d["input_ids"].to(device)
      attention_mask = d["attention_mask"].to(device)
      targets = d["targets"].to(device)

      target = targets[8:12]

      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )

      loss = loss_fn(outputs, targets)
      losses.append(loss.item())

      # _, preds = torch.max(outputs, dim=1)
      # one_correct_predictions += torch.sum(preds == targets)

      topk_values, topk_indices = torch.topk(outputs, k=3, dim=1)
      num_columns = topk_indices.shape[1]
      prediction_list = []

      for i in range(num_columns):
          column = topk_indices[8:12, i].tolist()
          prediction_list.append(column)
 
  return target.tolist(), prediction_list

  # return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
%%time
history = defaultdict(list)
best_accuracy = 0
for epoch in range(EPOCHS):
  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)
  train_acc, train_loss = train_epoch(
    model,
    train_data_loader,
    loss_fn,
    optimizer,
    DEVICE,
    scheduler,
    len(df_train)
  )
  print(f'Train loss {train_loss} accuracy {train_acc}')

  val_acc, val_loss, val_acc_one = eval_model(
    model,
    val_data_loader,
    loss_fn,
    DEVICE,
    len(df_val)
  ) #test_data_loader
  print(f'Val   loss {val_loss} accuracy {val_acc}')
  print(f'Val1  loss {val_loss} accuracy {val_acc_one}')

  test_acc, test_loss, test_acc_one = eval_model(
    model,
    test_data_loader,
    loss_fn,
    DEVICE,
    len(df_test)
  ) 
  print(f'Test   loss {test_loss} accuracy {test_acc}')
  print(f'Test1  loss {test_loss} accuracy {test_acc_one}')

  # if epoch == 10:
  #   target, pred_list = make_pred(
  #     model,
  #     val_data_loader,
  #     loss_fn,
  #     DEVICE,
  #     len(df_test)
  #   ) #test_data_loader
  #   print(f'Target emojis {target}')
  #   print(f'Predictions from model {pred_list}')

  print()
  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)
  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc


Epoch 1/10
----------


/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


KeyboardInterrupt: ignored

###Getting Prediciton

In [ ]:
  target, pred_list = make_pred(
    model,
    val_data_loader,
    loss_fn,
    DEVICE,
    len(df_test)
  ) #test_data_loader
  print(f'Target emojis {target}')
  print(f'Predictions from model {pred_list}')

Target emojis [39, 10, 33, 5]
Predictions from model [[39, 10, 12, 40], [16, 40, 7, 35], [0, 38, 8, 10]]


In [ ]:
Target_emojis = [39, 10, 33, 5]
Predictions_model = [[39, 10, 12, 40], [16, 40, 7, 35], [0, 38, 8, 10]]
print("Prediction Sample 1")
for i in range (4):
  print(f'Target prediction {products_list[Target_emojis[i]][1]} Most likely {products_list[Predictions_model[0][i]][1]} Second likely {products_list[Predictions_model[1][i]][1]} Third likely {products_list[Predictions_model[2][i]][1]}')

print()

Target_emojis = [7, 12, 45, 10]
Predictions_model = [[12, 34, 45, 9], [7, 12, 38, 5], [2, 4, 43, 11]]
print("Prediction Sample 2")
for i in range (4):
  print(f'Target prediction {products_list[Target_emojis[i]][1]} Most likely {products_list[Predictions_model[0][i]][1]} Second likely {products_list[Predictions_model[1][i]][1]} Third likely {products_list[Predictions_model[2][i]][1]}')


Prediction Sample 1
Target prediction 🎶 Most likely 🎶 Second likely 🤩 Third likely 💋
Target prediction 🎉 Most likely 🎉 Second likely 🥳 Third likely 🙏🏾
Target prediction 🤍 Most likely 😁 Second likely 🙏🏻 Third likely ‼️
Target prediction 🖤 Most likely 🥳 Second likely 💖 Third likely 🎉

Prediction Sample 2
Target prediction 🙏🏻 Most likely 😁 Second likely 🙏🏻 Third likely 😘
Target prediction 😁 Most likely 📸 Second likely 😁 Third likely ✨
Target prediction 😎 Most likely 😎 Second likely 🙏🏾 Third likely 💯
Target prediction 🎉 Most likely 🔥 Second likely 🖤 Third likely ❤️
